In [2]:
# %load drop_null.py
import os.path
from pyspark.context import SparkConf, SparkContext
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F

sc = SparkContext('local', 'bigdata_cw_v1')
spark = SparkSession(sc)
#from pyspark.sql import SparkSession
#spark = SparkSession.builder.enableHiveSupport().getOrCreate()

current_path = %pwd
filePath = os.path.join(os.path.abspath(current_path + os.path.sep + "."),'ieee-fraud-detection')
    
fileName_train_trans = 'train_transaction.csv'
file_train_trans = os.path.join(filePath, fileName_train_trans)

fileName_train_id = 'train_identity.csv'
file_train_id = os.path.join(filePath, fileName_train_id)
    
df = spark.read.options(header='true').csv(file_train_trans)
df2 = spark.read.options(header='true').csv(file_train_id)
train = df.join(df2, df.TransactionID == df2.TransactionID, 'left').drop(df2.TransactionID)
#train.drop('trainsid')

missing = train.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in train.columns])
null_counts = missing.collect()[0].asDict()
to_drop = [k for k, v in null_counts.items() if v > (590540*0.8)]
len(to_drop)
train.drop(*to_drop)


DataFrame[TransactionID: string, isFraud: string, TransactionDT: string, TransactionAmt: string, ProductCD: string, card1: string, card2: string, card3: string, card4: string, card5: string, card6: string, addr1: string, addr2: string, dist1: string, P_emaildomain: string, R_emaildomain: string, C1: string, C2: string, C3: string, C4: string, C5: string, C6: string, C7: string, C8: string, C9: string, C10: string, C11: string, C12: string, C13: string, C14: string, D1: string, D2: string, D3: string, D4: string, D5: string, D10: string, D11: string, D15: string, M1: string, M2: string, M3: string, M4: string, M5: string, M6: string, M7: string, M8: string, M9: string, V1: string, V2: string, V3: string, V4: string, V5: string, V6: string, V7: string, V8: string, V9: string, V10: string, V11: string, V12: string, V13: string, V14: string, V15: string, V16: string, V17: string, V18: string, V19: string, V20: string, V21: string, V22: string, V23: string, V24: string, V25: string, V26: st

In [3]:
from pyspark.sql.types import DoubleType

def remove_duplicate_columns(col_name, col_num, train):
    # cast column data type to double
    for i in range(1, col_num + 1):
        cName = col_name + str(i)
        train = train.withColumn(cName, F.col(cName).cast(DoubleType()))
    
    # calculate the correlation of C columns
    dup = []
    for i in range(1, col_num):
        col1 = col_name + str(i)
        col2 = col_name + str(i + 1)
        corr = train.corr(col1, col2)
        if (corr >= 0.95):
            dup.append(i + 1)

    # drop columns with duplicate data
    for i in range(len(dup)):
        cName = col_name + str(dup[i])
        train = train.drop(cName)
    return train
        
# C columns
train = remove_duplicate_columns('C', 14, train)
# D columns
train = remove_duplicate_columns('D', 15, train)
# V columns
train = remove_duplicate_columns('V', 339, train)

